# Lab 6 -- Decision Trees for Regression

Compared to last week, this is a very simple lab <span style="font-size:20pt;">😃</span> You'll have fun programming!

You will implement the **Classification and Regression Tree (CART)** algorithm from scratch.

The lab is broken down into the following pieces:

* Regression Criterion
* Creating Splits
* Buiding a Tree
* Making a prediction

## Exercise 1 -- Download and load the dataset

We will be using the usual Boston Housing dataset, which is available to download from ECLASS

* Download the file
* Read it and separate the target variable from the features.
* Make a 80/10/10 train/validation/test split

In [1368]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [1369]:
# your code here
data = pd.read_csv("BostonHousing.csv")
y = data["medv"].to_numpy()
X = data.drop("medv", axis=1).to_numpy()

The target variable will be as usual `MEDV`. Use the rest as features.

In [1370]:
# your code here
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5)

## Exercise 2 -- Optimization Criterion

For regression, a simple criterion to optimize is to minimize the sum of squared errors for a given region. This is, for all datapoints in a region with size, we minimize:

$$\sum_{i=1}^N(y_i - \hat{y})^2$$

where $N$ is the number of datapoits in the region and $\hat{y}$ is the mean value of the region for the target variable. 

Implement such a function using the description below.

Please, don't use an existing implementation, refer to the [book](https://www.statlearning.com/s/ISLRSeventhPrinting.pdf), and if you need help, ask questions!

In [1371]:
def regression_criterion(region):
    """
    Implements the sum of squared error criterion in a region
    
    Parameters
    ----------
    region : ndarray
        Array of shape (N,) containing the values of the target values 
        for N datapoints in the training set.
    
    Returns
    -------
    float
        The sum of squared error
        
    Note
    ----
    The error for an empty region should be infinity (use: float("inf"))
    This avoids creating empty regions
    """
    
    if len(region) == 0:
        return float("inf")
    
    mean_value = np.mean(region)
    sse = np.sum((region - mean_value) ** 2)
    
    return sse

In [1372]:
# test your code
rng = np.random.default_rng(0)
print(regression_criterion(rng.random(size=40)))
print(regression_criterion(np.ones(10)))
print(regression_criterion(np.zeros(10)))
print(regression_criterion(np.array([])))

3.6200679838629544
0.0
0.0
inf


## Exercise 3 -- Make a split

In [1373]:
def split_region(region, feature_index, tau):
    """
    Given a region, splits it based on the feature indicated by
    `feature_index`, the region will be split in two, where
    one side will contain all points with the feature with values 
    lower than `tau`, and the other split will contain the 
    remaining datapoints.
    
    Parameters
    ----------
    region : array of size (n_samples, n_features)
        a partition of the dataset (or the full dataset) to be split
    feature_index : int
        the index of the feature (column of the region array) used to make this partition
    tau : float
        The threshold used to make this partition
        
    Return
    ------
    left_partition : array
        indices of the datapoints in `region` where feature < `tau`
    right_partition : array
        indices of the datapoints in `region` where feature >= `tau` 
    """
    # your code here
    left_partition = np.where([region[:, feature_index] < tau])[1]
    right_partition = np.where([region[:, feature_index] >= tau])[1]

    return left_partition, right_partition

In [1374]:
l, r = split_region(X_train, 2, 10)
print(l.shape, r.shape)

(215,) (189,)


## Exercise 4 -- Find the best split

The strategy is quite simple (as well as inefficient), but it helps to reinforce the concepts.
We are going to use a greedy, exhaustive algorithm to select splits, selecting the `feature_index` and the `tau` that minimizes the Regression Criterion

In [1375]:
def get_split(X, y):
    """
    Given a dataset (full or partial), splits it on the feature of that minimizes the sum of squared error
    
    Parameters
    ----------
    X : array (n_samples, n_features)
        features 
    y : array (n_samples, )
        labels
    
    Returns
    -------
    decision : dictionary
        keys are:
        * 'feature_index' -> an integer that indicates the feature (column) of `X` on which the data is split
        * 'tau' -> the threshold used to make the split
        * 'left_region' -> array of indices where the `feature_index`th feature of X is lower than `tau`
        * 'right_region' -> indices not in `low_region`
    """
    # your code here
    min_sse = np.inf
    min_feature = 0
    min_tau = 0
    
    for each_feature in range(X.shape[1]):
        for each_tau in np.unique(X[:, each_feature]):
            left, right = split_region(X, each_feature, each_tau)

            sse1 = regression_criterion(y[left])
            sse2 = regression_criterion(y[right])
            
            total_sse = sse1 + sse2

            if total_sse < min_sse:
                min_sse = total_sse
                min_feature = each_feature
                min_tau = each_tau
                min_left = left
                min_right = right

    return {"feature_index":min_feature, "tau":min_tau, "left_region":min_left, "right_region":min_right}

In [1376]:
node_root = get_split(X_train[:15, :], y_train[:15])
print("Node Root:")
node_root

Node Root:


{'feature_index': 5,
 'tau': 6.943,
 'left_region': array([ 0,  1,  2,  4,  5,  6,  7,  8, 10, 13, 14], dtype=int64),
 'right_region': array([ 3,  9, 11, 12], dtype=int64)}

## Exercise 5 -- Recursive Splitting

The test above is an example on how to find the root node of our decision tree. The algorithm now is a greedy search until we reach a stop criterion. To find the actual root node of our decision tree, you must provide the whole training set, not just a slice of 15 rows as the test above.

The trivial stopping criterion is to recursively grow the tree until each split contains a single point (perfect node purity). If we go that far, it normally means we are overfitting.

You will implement these criteria to stop the growth:

* A node is a leaf if:
    * It has less than `min_samples` datapoints
    * It is at the `max_depth` level from the root (each split creates a new level)
    * The criterion is `0`



In [1377]:
def recursive_growth(node, min_samples, max_depth, current_depth, X, y):
    """
    Recursively grows a decision tree.
    
    Parameters
    ----------
    node : dictionary
        If the node is terminal, it contains only the "value" key, which determines the value to be used as a prediction.
        If the node is not terminal, the dictionary has the structure defined by `get_split`
    min_samples : int
        parameter for stopping criterion if a node has <= min_samples datapoints
    max_depth : int
        parameter for stopping criterion if a node belongs to this depth
    depth : int
        current distance from the root
    X : array (n_samples, n_features)
        features (full dataset)
    y : array (n_samples, )
        labels (full dataset)
    
    Notes
    -----
    To create a terminal node, a dictionary is created with a single "value" key, with a value that
    is the mean of the target variable
    
    'left' and 'right' keys are added to non-terminal nodes, which contain (possibly terminal) nodes 
    from higher levels of the tree:
    'left' corresponds to the 'left_region' key, and 'right' to the 'right_region' key
    """
    # vai checar as condições de parada
    if len(y) <= min_samples or current_depth >= max_depth or regression_criterion(y) == 0:
        node["value"] = np.mean(y)
        return
    
    # vai partir os dados
    split = get_split(X, y)
    node.update(split)
    
    # crecer recursivamente para esquerda
    left_indices = node["left_region"]
    if len(left_indices) == 1:
        node["left"] = {"value": np.mean(y[left_indices])}
    else:
        node["left"] = {}
        recursive_growth(node["left"], min_samples, max_depth, current_depth + 1, X[left_indices], y[left_indices])
    
    # crescer recursivamente para a direita
    right_indices = node["right_region"]
    if len(right_indices) == 1:
        node["right"] = {"value": np.mean(y[right_indices])}
    else:
        node["right"] = {}
        recursive_growth(node["right"], min_samples, max_depth, current_depth + 1, X[right_indices], y[right_indices])

In [1378]:
k = 20
test_root = get_split(X_train[:k, :], y_train[:k])
recursive_growth(test_root, 5, 4, 1, X_train[:k, :], y_train[:k])

In [1379]:
# fill in the gaps with your code
min_samples = 3
max_depth = 6
root = get_split(X, y)
recursive_growth(root, min_samples, max_depth, 0, X_train[:20, :], y_train[:20])

Below we provide code to visualise the generated tree!

In [1380]:
def print_tree(node, depth):
    if 'value' in node.keys():
        print('.  '*(depth-1), f"[{node['value']}]")
    else:
        print('.  '*depth, f'X_{node["feature_index"]} < {node["tau"]}')
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)

In [1381]:
print_tree(root, 0)

 X_12 < 5.7
.   X_5 < 7.686
.   [36.46666666666666]
.   [48.9]
.   X_4 < 0.538
.  .   X_4 < 0.489
.  .  .   X_5 < 6.417
.  .  .   [20.266666666666666]
.  .  .   [24.2]
.  .  .   X_1 < 12.5
.  .  .   [29.566666666666666]
.  .  .   [27.1]
.  .   X_0 < 38.3518
.  .  .   X_0 < 0.77299
.  .  .   [11.9]
.  .  .  .   X_0 < 8.05579
.  .  .  .   [17.9]
.  .  .  .   [14.2]
.  .   [5.0]


# Exercise 6 -- Make a Prediction
Use the a node to predict the class of a compatible dataset

In [1382]:
def predict_sample(node, sample):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    sample : array of size (n_features,)
        a sample datapoint
    """
    # your code here
    while "value" not in node:
        if sample[node["feature_index"]] < node["tau"]:
            node = node["left"]
        else:
            node = node["right"]
    return node["value"]
        
def predict(node, X):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    X : array of size (n_samples, n_features)
        n_samples predictions will be made
    """
    # your code here
    y_temp = np.zeros((X.shape[0], 1))
    for each_datapoint in range(X.shape[0]):
        y_temp[each_datapoint] = predict_sample(node, X[each_datapoint,:])
    return y_temp

In [1383]:
predict(root, X[:15,:])

array([[36.46666667],
       [24.2       ],
       [36.46666667],
       [36.46666667],
       [36.46666667],
       [36.46666667],
       [27.1       ],
       [27.1       ],
       [27.1       ],
       [27.1       ],
       [27.1       ],
       [27.1       ],
       [27.1       ],
       [11.9       ],
       [11.9       ]])

Now use the functions defined above to calculate the RMSE of the validation set. 
* Try first with `min_samples=20` and `max_depth=6` (for this values you should get a validation RMSE of ~8.8)

Then, experiment with different values for the stopping criteria.

In [1384]:
# your code here
min_samples = 20
max_depth = 6
root_node = get_split(X_train, y_train)
recursive_growth(root_node, min_samples, max_depth, 0, X_train, y_train)
y_pred = predict(root_node, X_val)

In [1385]:
SSE = np.sum(np.power(y_pred - y_val, 2))
SSE

375096.8237136695

In [1386]:
RMSE = np.sqrt(np.mean(np.power(y_pred - y_val, 2)))
RMSE

12.008852682835395